In [6]:
from __future__ import print_function
import sklearn.utils
from sklearn import svm as SVM
from numpy.random import seed
from numpy.random import randn
from sklearn.metrics import mean_squared_error as MSE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn.decomposition import PCA, KernelPCA
import collections
#from imblearn.over_sampling import SMOTE
from scipy.stats import shapiro,anderson,normaltest
#import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier
from numpy import loadtxt
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
import requests
import lxml
from bs4 import BeautifulSoup as bs
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from random import choice, shuffle
from collections import defaultdict
import csv
import pandas as pd
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_iris
import pandasql as pdsql
from py2neo import Graph
get_ipython().run_line_magic('matplotlib', 'inline')

ImportError: No module named 'requests'

In [360]:
def Is_Gaussian_Distribution(Feature):
    alpha = 0.05
    stat,p_shapiro=shapiro(Feature)
    stat,p_DAgostinos=normaltest(Feature)
    if p_shapiro > alpha and p_DAgostinos > alpha :
        return True
    else:
        return False

In [361]:
def RobustScaler(Col_Name,Col_Series):

    Old_Col_Name=Col_Name
    New_Col_Name=Col_Name+'_RobustNormalized'

    # Create x, where x the 'Name_Of_Column' column's values as floats
    x=Col_Series.values.astype(float).reshape(-1, 1)
    # Create a minimum and maximum processor object
    RobustScaler = preprocessing.RobustScaler()
    # Run the normalizer on the dataframe
    scaled = RobustScaler.fit_transform(x)
    # add column to file
    file[New_Col_Name] = scaled
    
    del file[Col_Name]
    Numeric_List.remove(Col_Name)
    Numeric_List.append(New_Col_Name)

    return 

In [362]:
def StandardScaler(Col_Name,Col_Series):

    Old_Col_Name=Col_Name
    New_Col_Name=Col_Name+'_normalized'

    # Create x, where x the 'Name_Of_Column' column's values as floats
    x=Col_Series.values.astype(float).reshape(-1, 1)
    # Create a minimum and maximum processor object
    StandardScaler = preprocessing.StandardScaler()
    # Run the normalizer on the dataframe
    scaled = StandardScaler.fit_transform(x)
    # add column to file
    file[New_Col_Name] = scaled
    
    del file[Col_Name]
    Numeric_List.remove(Col_Name)
    Numeric_List.append(New_Col_Name)

    return 

In [363]:
def class_size(data_frame,target):
    values = file[target].value_counts().keys().tolist()
    counts = file[target].value_counts().tolist()
    if counts[0]>counts[1]:
        majority_class=values[0]
        majority_count=counts[0]
        minority_class=values[1]
        minority_count=counts[1]
    else:
        majority_class=values[1]
        majority_count=counts[1]
        minority_class=values[0]
        minority_count=counts[0]
        
    return  majority_class,majority_count,minority_class,minority_count  

In [364]:
def Dimensionality_Reduction(data,PCA_type,Numeric_List):
    if PCA_type==1:
        Dim_Reduction = KernelPCA(kernel="rbf", gamma=15, n_components=2)
        Dim_Reduction_result = Dim_Reduction.fit_transform(data[Numeric_List].values)
    else:
        Dim_Reduction = PCA(n_components=0.95)
        Dim_Reduction_result = Dim_Reduction.fit_transform(data[Numeric_List].values)
        
        
    dataframe = pd.DataFrame(Dim_Reduction_result)
    dataframe.head()
    file=pd.merge(data,dataframe,left_index=True,right_index=True,how='inner')
    file=file.drop(Numeric_List, axis=1)
    return file
        

In [365]:
def Scale_Numeric_Features(Numeric_dataset):

    for col in Numeric_dataset:
        if Is_Gaussian_Distribution(Numeric_dataset[col]):
            StandardScaler(col,Numeric_dataset[col])
        else: 
            RobustScaler(col,Numeric_dataset[col])

In [368]:
def split_sample(sample_type,data,target):
    
    if sample_type==0:
        #Split data
        X_train, X_test, y_train, y_test = split(data.drop(target, axis=1), data[target])
        return X_train, X_test, y_train, y_test
    
    if sample_type==1:
    
        #OverSampling
        file=data
        list_of_names=data.drop(target, axis=1).columns
        X_train, X_test, y_train, y_test = split(file.drop(target, axis=1), file[target])
        sm = SMOTE(random_state=12, ratio = 1.0)
        X_train, y_train  = sm.fit_sample(X_train, y_train)
        y_train=pd.Series(y_train)
        X_train = pd.DataFrame.from_records(X_train)
        X_train.columns = list_of_names
        return X_train, X_test, y_train, y_test
    
    if sample_type==2:
        #Under Sampling
        # split file to train and test before Sampling
        file = sklearn.utils.shuffle(data)
        total_rows = len(file)
        head=total_rows*0.7
        tail=total_rows*0.3
        train_file=file.head(int(head))
        test_file=file.tail(int(tail))
        file=train_file


        #Under Sampling
        majority_class,majority_count,minority_class,minority_count=class_size(file,target)
        minority_extract=file[file[target]== minority_class]

        sample_size = minority_count# sum of minority class
        majority_indices=file[file[target]== majority_class].index #  the majority class df
        random_indices = np.random.choice(majority_indices, sample_size, replace=False)
        majority_sample = file.loc[random_indices]
        file=pd.concat([minority_extract,majority_sample])

        target_count = file[target].value_counts()
        #print('Class 0:', target_count[0],round(target_count[0]/target_count.sum(),2))
        #print('Class 1:', target_count[1],round(target_count[1]/target_count.sum(),2))
        #print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

        #target_count.plot(kind='bar', title='Count (target)');


        #Split data
        X_train=file.drop(target, axis=1)
        y_train=file[target]
        X_test=test_file.drop(target, axis=1)
        y_test=test_file[target]
        return X_train, X_test, y_train, y_test


In [369]:

#read data from csv
file = pd.read_csv(r"C:\Users\Harel\Desktop\train_titanic.csv", encoding='utf-8')
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(712) + 50
file['data']=data
#initiate params
Numeric_List=['Age', 'SibSp','Parch','Fare','data']
PCA_type=1
sample_type=2
Numeric_dataset = file.loc[:, Numeric_List]
Categorical_list=['Sex','Pclass','Embarked']
target='Survived'
# number of iteration for model
range_param=2

file.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,data
0,3,male,22,1,0,7.2500,S,0,58.121727
1,1,female,38,1,0,71.2833,C,1,46.941218
2,3,female,26,0,0,7.9250,S,1,47.359141
3,1,female,35,1,0,53.1000,S,1,44.635157
4,3,male,35,0,0,8.0500,S,0,54.327038


In [352]:
# Scale Numeric Features
Scale_Numeric_Features(Numeric_dataset)

In [353]:
#Dimensionality_Reduction
file=Dimensionality_Reduction(file,PCA_type,Numeric_List)

In [354]:
# One hot incoding of Categorical Features
file=pd.get_dummies(file, columns=Categorical_list) 

In [375]:
file.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,data
0,3,male,22,1,0,7.2500,S,0,58.121727
1,1,female,38,1,0,71.2833,C,1,46.941218
2,3,female,26,0,0,7.9250,S,1,47.359141
3,1,female,35,1,0,53.1000,S,1,44.635157
4,3,male,35,0,0,8.0500,S,0,54.327038


In [370]:
#Split data
X_train, X_test, y_train, y_test = split_sample(sample_type,file,target)

In [357]:
# base models
clf1 = LogisticRegression()
clf2 = DecisionTreeClassifier(max_depth=5)
clf3 = KNeighborsClassifier()
#clf4= SVM.SVC(kernel='linear', C = 1.0)
#clf5 = SVM.SVC(kernel='poly', degree=8) #Non-linearly Separable Data

classifiers_base = [('LR', clf1), ('DT', clf2), ('KN', clf3)]#, ('SVM_L', clf4), ('SVM_K', clf5)

# Bagging meta-estimator(BME)
clf6 = BaggingClassifier(base_estimator=clf1,n_estimators=100)
clf7 = BaggingClassifier(base_estimator=clf2,n_estimators=100)
clf8 = BaggingClassifier(base_estimator=clf3,n_estimators=100)
#clf9 = BaggingClassifier(base_estimator=clf4,n_estimators=100)
#clf10 = BaggingClassifier(base_estimator=clf5,n_estimators=100)
                               
classifiers_Bagging = [('BME_LR', clf6), ('BME_DT', clf7), ('BME_KN', clf8)]    #, ('BME_SVM_L', clf9), ('BME_SVM_K', clf10)

 #Random Forest

clf11 = RandomForestClassifier(random_state=1)
clf15 = RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini')
clf16 = RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy')



classifiers_Random_Forest = [('RF', clf11),('RF_gini', clf15),('RF_entropy', clf16)]

#Boosting

clf12 = AdaBoostClassifier(random_state=1)
clf13= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
clf14=xgb.XGBClassifier(random_state=1,learning_rate=0.01)

classifiers_Boosting = [('AdaBoost', clf12), ('GradientBoost', clf13), ('XGB', clf14)]  

#Cmbine basic classifiers
basic_classifiers=classifiers_base+classifiers_Bagging+classifiers_Random_Forest+classifiers_Boosting


# Voting Classifiers
clf17 = VotingClassifier(estimators=basic_classifiers, voting='hard')
clf18 = VotingClassifier(estimators=basic_classifiers, voting='soft')

classifiers_Voting = [('VotingClassifier_hard', clf17), ('VotingClassifier_soft', clf18)]  

# fit, predict and and print results for all the classifiers
classifiers=basic_classifiers+classifiers_Voting
train_results = y_train.to_frame()
train_results_proba = y_train.to_frame()
list_to_df=[]
temp_dic={}
temp_list=[]
#range_param=1
for clf_name, clf in classifiers:
    try: 
        clf.fit(X_train, y_train)
        #train_results_proba[clf_name] = clf.predict_proba(X_train)[:,1]
        #train_results[clf_name] = clf.predict(X_train)
        temp_dic={}
        temp_list=[] 
        Precision=0
        recall=0
        f1score=0
        for x in range(range_param):
            clf.fit(X_train, y_train)
            Precision=Precision+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[0]
            recall=recall+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[1]
            f1score=f1score+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[2]

        temp_dic['Avg_Precision']=Precision/range_param
        temp_dic['Avg_recall']=recall/range_param
        temp_dic['Avg_f1score']=f1score/range_param
        temp_dic['Classifier']=clf_name
        temp_list=temp_dic
        list_to_df.append(temp_list) 
    except:

        
        #train_results[clf_name] = clf.predict(X_train)
        temp_dic={}
        temp_list=[] 
        Precision=0
        recall=0
        f1score=0
        for x in range(range_param):
            clf.fit(X_train, y_train)
            Precision=Precision+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[0]
            recall=recall+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[1]
            f1score=f1score+precision_recall_fscore_support(y_test,clf.predict(X_test), average='micro')[2]
        temp_dic['Avg_Precision']=Precision/range_param
        temp_dic['Avg_recall']=recall/range_param
        temp_dic['Avg_f1score']=f1score/range_param
        temp_dic['Classifier']=clf_name
        temp_list=temp_dic
        list_to_df.append(temp_list)


Report = pd.DataFrame(list_to_df)
Report.sort_values('Avg_Precision',ascending =False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

,Avg_Precision,Avg_f1score,Avg_recall,Classifier
11,0.788732,0.788732,0.788732,XGB
10,0.779343,0.779343,0.779343,GradientBoost
5,0.772300,0.772300,0.772300,BME_KN
2,0.769953,0.769953,0.769953,KN
4,0.769953,0.769953,0.769953,BME_DT
0,0.755869,0.755869,0.755869,LR
3,0.755869,0.755869,0.755869,BME_LR
12,0.755869,0.755869,0.755869,VotingClassifier_hard
1,0.730047,0.730047,0.730047,DT
13,0.730047,0.730047,0.730047,VotingClassifier_soft


In [334]:

# Training Data
tf.reset_default_graph()
train_X = X_train.values
train_Y = np.expand_dims(y_train.values, axis=1)



# Test Data
test_X = X_test.values
test_Y = np.expand_dims(y_test.values, axis=1)




# Parameters
learning_rate = 0.001
training_epochs = 5000
display_step = 600
num_inputs = 10
num_inputs_hidden=5
batch_size = 50
train_len = np.int(np.floor(len(train_Y) / batch_size))




n_samples = train_X.shape[0]




# Placeholders
X = tf.placeholder(tf.float32, [None, num_inputs], name='X')
Y = tf.placeholder(tf.float32, [None, 1], name='Y')

# In[894]:


# Set model weights

W = tf.get_variable("W", [num_inputs, num_inputs_hidden], initializer=tf.contrib.layers.xavier_initializer(uniform=True))
b = tf.get_variable("b", [num_inputs_hidden], initializer=tf.zeros_initializer())




W1 = tf.get_variable("W1", [num_inputs_hidden, 1], initializer=tf.contrib.layers.xavier_initializer(uniform=True))
b1 = tf.get_variable("b1", [1], initializer=tf.zeros_initializer())
global_step = tf.Variable(0, trainable=False, name='global_step')

# In[896]:



# Construct a linear model
hidden = tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(X, W), b))
pred = tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(hidden, W1), b1))

# In[897]:


# Mean squared error
cost = tf.losses.mean_squared_error(pred, Y)

# In[898]:


# Gradient descent
#  Note, minimize() knows to modify W and b because Variable objects are trainable=True by default
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# In[899]:


# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# In[900]:


# with tf.Session() as sess:
sess = tf.Session()
# Run the initializer
sess.run(init)
rnd_perm = np.arange(len(train_Y))
for epoch in range(training_epochs):

    # scramble training data
    rnd_perm = np.random.permutation(rnd_perm)
    train_X = train_X[rnd_perm, :]
    train_Y = train_Y[rnd_perm]

    for step in range(0, train_len):
        batch_X = train_X[step * batch_size:(step + 1) * batch_size, :]
        batch_Y = train_Y[step * batch_size:(step + 1) * batch_size]

        sess.run(optimizer, feed_dict={X: batch_X, Y: batch_Y})

        # Display logs per epoch step
        if (step % display_step) == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
           # print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c), "W=", sess.run(W), "b=", sess.run(b))
            #print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c))

res_Y = sess.run(pred, feed_dict={X: test_X})
res_Y = np.array(np.round(res_Y), dtype=int)
acc = np.sum(test_Y == res_Y) / test_Y.shape[0]

#print('Final Accuracy = ' + str(acc))
#print('Final Precision = ' + str(precision_score(test_Y, res_Y)))

print(classification_report(y_test, res_Y))
tetot = sess.run([W, b])
##print (tetot)
## print(res_Y)
##print(sess.run([W1,b1]))
sess.close()

              precision    recall  f1-score   support

           0       0.67      0.98      0.79       126
           1       0.93      0.29      0.44        87

   micro avg       0.70      0.70      0.70       213
   macro avg       0.80      0.64      0.62       213
weighted avg       0.77      0.70      0.65       213

